# Analyse de LMGEC

> Article analysé : **LMGEC: Simultaneous Linear Multi-view Attributed Graph Representation Learning and Clustering**, WSDM 2023.


---


## Problématique centrale  :  
LMGEC répond à la problématique suivante :
“Comment apprendre des représentations partagées et effectuer un clustering efficace sur des graphes multi-vues attribués, tout en garantissant simplicité, rapidité, et robustesse face à l’hétérogénéité des vues ?”
>

![Alt text](problem.png)


## 📅 1. Méthodologie : quel type de fusion ?

LMGEC repose sur une **fusion linéaire pondée tardive** des vues. 

- Chaque vue est d'abord **filtrée localement** (1-hop) pour lisser les attributs : $$ H_v = S_v X_v $$
- Une **pondération adaptative** des vues est appliquée via des poids $$ \alpha_v $$, calculés dynamiquement.
- Les embeddings obtenus sont projetés et **fusionnés dans un espace commun** pour effectuer le clustering.

> **Type de fusion :** tardive + pondération adaptative (soft fusion)

---

## 🔍 2. Hypothèses sur les vues

- ✅ Mêmes nœuds dans toutes les vues
- ✅ Vues potentiellement **très hétérogènes** (topologies ou attributs)
- ❌ Pas de traitement spécial des vues **manquantes** ou désalignées
- ✅ Le modèle peut atténuer les vues bruitées via la pondération $$ \alpha_v $$

---

## 📊 3. Modèles mathématiques

- **Filtrage de chaque vue :** $$ H_v = S_v X_v $$, où $$ S_v = \tilde{D}^{-1} (\tilde{A}_v) $$ avec self-loops.

- **Objectif :**
$$
\min_{G, F, W_1,\dots,W_V} \sum_{v=1}^{V} \alpha_v \| H_v - G F W_v^\top \|^2
$$
Avec :
  - $ G \in \{0,1\}^{n \times k} $ : clustering (soft or hard)
  - $ W_v \in \mathbb{R}^{d \times f} $, $ W_v W_v^\top = I $

- **Pondération des vues :**
$$
\alpha_v = \text{softmax}\left(-\frac{I_v}{\tau}\right), \quad I_v = \| H_v - G_v F_v \|
$$

> Optimisation par **Bloc Coordinate Descent**

> 
![Alt text](recon.png)
---

## 📈 4. Types de données utilisées

- **Topologies différentes, mêmes features :** ACM, DBLP, IMDB
- **Mêmes topologies, features différentes :** Amazon Photos
- **Topologies + features différentes :** Wiki

> LMGEC couvre **tous les cas multi-vues usuels**
> 
![Alt text](topo.png)
---

## 📊 5. Métriques d’évaluation

Les performances sont mesurées avec 4 métriques standards en clustering :

| Métrique | Description |
|----------|-------------|
| **NMI** | Normalized Mutual Information |
| **ARI** | Adjusted Rand Index |
| **ACC** | Accuracy (appliquée au clustering) |
| **F1-score** | Précision + rappel |

---


# 📊 Métriques d’évaluation pour le clustering non supervisé

Lorsque j’évalue un algorithme de clustering (comme LMGEC), je dois utiliser des métriques quantitatives pour mesurer à quel point les clusters trouvés sont cohérents avec les classes réelles (si elles sont disponibles). Voici les principales métriques utilisées :

---

| **Métrique** | **Nom complet** | **Ce qu’elle mesure** | **Valeurs typiques** |
|--------------|------------------|------------------------|-----------------------|
| **NMI** | Normalized Mutual Information | Le degré de similarité entre les clusters prédits et les vraies classes, basé sur l’information partagée | Entre 0 (aucune info) et 1 (parfait) |
| **ARI** | Adjusted Rand Index | À quel point deux partitions (clusters vs classes) sont en accord, corrigé pour le hasard | Entre -1 (pire que hasard) et 1 (parfait accord) |
| **ACC** | Clustering Accuracy | Pourcentage d’exemples correctement classés, après réassignation optimale des labels | Entre 0 et 1 (ou 0% à 100%) |
| **F1-score** | F1 Measure (harmonique précision / rappel) | Moyenne harmonique entre la précision (exactitude des clusters) et le rappel (couverture des vraies classes) | Entre 0 et 1 |

---

## 🧠 Détails complémentaires

### 🔹 NMI (Normalized Mutual Information)
- Formule :
$$
\text{NMI}(Y, C) = \frac{2 \, I(Y; C)}{H(Y) + H(C)}
$$
avec \( I(Y; C) \) l'information mutuelle entre la partition vraie \( Y \) et prédite \( C \).
- Avantage : **invariante aux permutations de labels**.

### 🔹 ARI (Adjusted Rand Index)
- Tient compte du **nombre de paires** correctement regroupées ou séparées.
- Corrige le **Rand Index** pour compenser le regroupement aléatoire.
- Avantage : bonne robustesse même avec des déséquilibres de classe.

### 🔹 ACC (Accuracy)
- Mesure directe et intuitive.
- Requiert une **permutation optimale des clusters** (car les labels peuvent être inversés).

### 🔹 F1-score
- Combine deux notions :
  - **Précision** : parmi les éléments du cluster, combien sont bien classés ?
  - **Rappel** : parmi les vrais éléments d’une classe, combien sont capturés ?
- L’équilibre entre les deux est utile surtout pour les classes déséquilibrées.

---

> En résumé : utiliser plusieurs métriques permet d’avoir une vue plus complète de la qualité du clustering. NMI et ARI sont les plus stables pour comparer les méthodes, tandis que ACC et F1 donnent une interprétation plus intuitive des performances.





# 🎯 Contribution originale et évaluation critique de LMGEC

## 🧩 Quelle est la **contribution originale** du papier ?

L’article propose **LMGEC**, un modèle linéaire simple et efficace pour :

- Réaliser **simultanément** l’apprentissage de représentation et le clustering sur des graphes multi-vues attribués.
- Offrir une **formulation unifiée** intégrant :
  - une étape de **propagation locale (1-hop)** par un filtre de graphe linéaire,
  - un **mécanisme de pondération adaptative des vues** (softmax sur l’inertie),
  - un objectif combiné de **reconstruction + clustering**.
- Être **générique**, applicable à :
  - plusieurs graphes avec une même matrice de features,
  - plusieurs matrices de features sur un seul graphe,
  - ou un mélange des deux (cas du dataset Wiki).
- Être **beaucoup plus rapide** que les modèles existants tout en offrant des performances comparables, voire meilleures.
- Fournir une **analyse mathématique et expérimentale approfondie** ainsi que le **code open-source**.

## ✅ **Points forts** de LMGEC

| Atout | Détail |
|-------|--------|
| ✅ Simplicité | Formulation linéaire claire et interprétable |
| ✅ Efficacité | Temps d'entraînement **jusqu'à 10× plus rapide** que les méthodes GCN ou autoencoder |
| ✅ Robustesse | Capacité à ignorer les vues peu informatives via le mécanisme d'attention/inertie |
| ✅ Généralité | Supporte différents types de graphes multi-vues sans contraintes |
| ✅ Formulation unifiée | Apprentissage de représentation + clustering dans un même objectif |
| ✅ Reproductibilité | Code disponible en open-source et résultats détaillés sur 5 benchmarks |

## ⚠️ **Limites** de LMGEC

| Limite | Détail |
|--------|--------|
| ❌ Méthode linéaire | Ne capture pas les non-linéarités complexes, contrairement aux modèles deep |
| ❌ Pas de gestion des vues manquantes | Chaque vue est supposée complète et bien alignée |
| ❌ Risque de sur-lissage évité uniquement via un filtrage 1-hop | Ce choix reste rigide dans certains cas |
| ❌ Pas de mécanisme d'apprentissage end-to-end avec supervision éventuelle | Modèle strictement non supervisé |
| ❌ Pas de mécanisme explicite de fusion dynamique | Le poids est fixé après initialisation (pas appris pendant optimisation) |

## 🧠 Conclusion
LMGEC se positionne comme une **alternative simple, rapide et robuste** aux méthodes complexes basées sur GCN ou autoencodeurs. Il est particulièrement pertinent dans des contextes contraints en ressources ou nécessitant une interprétabilité forte.

Cependant, pour des cas très non-linéaires ou avec des données partiellement alignées, des méthodes plus expressives comme les modèles à attention ou graph contrastif peuvent être préférées.

## 🌐 Récapitulatif Synthétique

| Élément | Déscription |
|--------|-------------|
| **Fusion** | Tardive, linéaire, pondération adaptative |
| **Hypothèses sur les vues** | Mêmes nœuds, hétérogénéité supportée |
| **Formulation** | Linéaire, objectif joint reconstruction + clustering |
| **Données** | Multi-vues topologiques, attributaires ou mixtes |
| **Métriques** | NMI, ARI, ACC, F1 |

---